In [38]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import json
import copy

In [110]:
with open("./data/vgc2023s1/team-pokemon.json", "r") as f:
  team_pokemon = json.load(f)

team_pokemon_df = pd.DataFrame(team_pokemon)
team_pokemon_df

,team_id,rank,species,showdown_pokedex_name,item,ability,tera_type,nature,move_1,move_2,move_3,move_4,type_1,type_2,base_hp,base_atk,base_def,base_spa,base_spd,base_spe
0,284,1,Dondozo,dondozo,leftovers,unaware,dragon,jolly,wave-crash,substitute,order-up,protect,water,None,150,100,115,65,65,35
1,284,1,Tatsugiri,tatsugiri,choice-scarf,commander,water,modest,draco-meteor,muddy-water,icy-wind,sleep-talk,dragon,water,68,50,60,120,95,82
2,284,1,Gholdengo,gholdengo,choice-specs,good-as-gold,steel,modest,make-it-rain,shadow-ball,thunderbolt,power-gem,steel,ghost,87,60,95,133,91,84
3,284,1,Murkrow,murkrow,eviolite,prankster,ghost,calm,tailwind,foul-play,quash,haze,dark,flying,60,85,42,85,42,91
4,284,1,Dragonite,dragonite,assault-vest,inner-focus,flying,adamant,extreme-speed,stomping-tantrum,tera-blast,low-kick,dragon,flying,91,134,95,100,100,80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,283,32,Kingambit,kingambit,assault-vest,defiant,flying,nature,kowtow-cleave,low-kick,sucker-punch,iron-head,dark,steel,100,135,120,60,85,50
380,283,32,Pelipper,pelipper,life-orb,drizzle,fairy,nature,hurricane,wide-guard,hydro-pump,tailwind,water,flying,60,50,100,95,70,65
381,283,32,Gholdengo,gholdengo,leftovers,good-as-gold,steel,nature,make-it-rain,protect,nasty-plot,shadow-ball,steel,ghost,87,60,95,133,91,84
382,283,32,Mimikyu,mimikyu,safety-goggles,disguise,fairy,nature,play-rough,curse,shadow-sneak,trick-room,ghost,fairy,55,90,80,50,105,96


In [111]:
team_pokemon_consolidated = pd.get_dummies(team_pokemon_df["species"])
team_pokemon_consolidated["team_id"] = team_pokemon_df["team_id"]
team_pokemon_consolidated = team_pokemon_consolidated.groupby("team_id").sum().reset_index().drop(columns=["team_id"])

# cols = team_pokemon_consolidated.columns.to_list()
# cols = cols[-1:] + cols[:-1]

# team_pokemon_consolidated = team_pokemon_consolidated[cols]
team_pokemon_consolidated.sum(axis=1).mean()

6.0

In [112]:
frequent_teammates = apriori(team_pokemon_consolidated, min_support=0.1, use_colnames=True)

In [113]:
rules = association_rules(frequent_teammates, metric="lift", min_threshold=1)
rules.sort_values("lift", ascending=False)[rules["lift"] > 1].head(10)

C:\Users\kash\AppData\Local\Temp\ipykernel_3548\3483819449.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  rules.sort_values("lift", ascending=False)[rules["lift"] > 1].head(10)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
282,"(Meowscarada, Gholdengo, Dondozo)",(Tatsugiri-Stretchy),0.125000,0.140625,0.109375,0.875000,6.222222,0.091797,6.875000,0.959184
295,(Tatsugiri-Stretchy),"(Meowscarada, Gholdengo, Dondozo)",0.140625,0.125000,0.109375,0.777778,6.222222,0.091797,3.937500,0.976623
281,(Palafin),"(Amoonguss, Salamence, Pelipper)",0.171875,0.140625,0.125000,0.727273,5.171717,0.100830,3.151042,0.974057
181,(Palafin),"(Salamence, Pelipper)",0.171875,0.140625,0.125000,0.727273,5.171717,0.100830,3.151042,0.974057
107,(Palafin),"(Amoonguss, Salamence)",0.171875,0.140625,0.125000,0.727273,5.171717,0.100830,3.151042,0.974057
274,"(Palafin, Pelipper)","(Amoonguss, Salamence)",0.171875,0.140625,0.125000,0.727273,5.171717,0.100830,3.151042,0.974057
276,"(Amoonguss, Palafin)","(Salamence, Pelipper)",0.171875,0.140625,0.125000,0.727273,5.171717,0.100830,3.151042,0.974057
102,"(Amoonguss, Salamence)",(Palafin),0.140625,0.171875,0.125000,0.888889,5.171717,0.100830,7.453125,0.938636
180,"(Salamence, Pelipper)",(Palafin),0.140625,0.171875,0.125000,0.888889,5.171717,0.100830,7.453125,0.938636
268,"(Amoonguss, Salamence, Pelipper)",(Palafin),0.140625,0.171875,0.125000,0.888889,5.171717,0.100830,7.453125,0.938636


# Analysis

The high lift on Tatsugiri from Dondozo-containing sub-teams is unsurprising, because of Tatsugiri's unique ability _Commander_ which allows it to go inside an ally Dondozo's mouth and raise all of its stats. It's only natural that there would be a strong association between the two.

The association between Palafin and Salamence, on the other hand, is not so obviously explained. So, I decided to do some digging to find out why, and I think I've got it: Meowscarada and/or Amoonguss.

Palafin, being a Water type, is weak to Grass and Electric type moves. One of the most commonly used Pokemon in VGC 2023 Series 1 is Meowscarada (where it appeared around 44% of the time). Meowscarada has very high Speed and Attack stats, which means it's most likely to be used as a physical sweeper. Salamence, being a dual Dragon/Flying type, has a double resistance to Grass type moves, thereby allowing it to take Meowscarada's powerful signature move, Flower Trick, which always results in a critical hit and therefore does 115 base damage, ignoring the attacker's negative stat changes as well as the defender's positive stat changes.

Additionaly, Salamence has impressive Speed, Attack, and Special Attack stats of its own, enabling it to take out any overconfident Meowscarada it encounters. It is also neutrally affected by Electric type moves, and Palafin, in turn, covers Salamence's double Ice type and single Rock, Dragon, and Fairy type weaknesses.

Amoonguss is just very common in general, appearing in around 63% of all teams in VGC 2023 Series 1. It is also a Grass type, has an enormous 114 base HP stat, and has access to Spore, a move that can make opposing Pokemon fall asleep for one to three turns, making it quite dangerous. Salamence, being a very fast, very hard hitting Grass type destroyer, is presumably helpful in putting enemy Amoongusses (or is the plural just "Amoonguss"?) in their place.

That said, I don't think Amoonguss is as good as Meowscarada as an explanation for the Palafin-Salamence association, simply because Amoonguss isn't an attacker as much as a status inflicter, so Salamence can be replaced with other Pokemon to counter Amoonguss.

In [114]:
rules.sort_values("support", ascending=False)[rules["lift"] > 1].head(10)

C:\Users\kash\AppData\Local\Temp\ipykernel_3548\2391800750.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  rules.sort_values("support", ascending=False)[rules["lift"] > 1].head(10)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(Amoonguss),(Gholdengo),0.437500,0.625000,0.281250,0.642857,1.028571,0.007812,1.050000,0.049383
1,(Gholdengo),(Amoonguss),0.625000,0.437500,0.281250,0.450000,1.028571,0.007812,1.022727,0.074074
27,(Gholdengo),(Meowscarada),0.625000,0.437500,0.281250,0.450000,1.028571,0.007812,1.022727,0.074074
26,(Meowscarada),(Gholdengo),0.437500,0.625000,0.281250,0.642857,1.028571,0.007812,1.050000,0.049383
2,(Amoonguss),(Kingambit),0.437500,0.375000,0.265625,0.607143,1.619048,0.101562,1.590909,0.679739
3,(Kingambit),(Amoonguss),0.375000,0.437500,0.265625,0.708333,1.619048,0.101562,1.928571,0.611765
40,(Pelipper),(Kingambit),0.265625,0.375000,0.250000,0.941176,2.509804,0.150391,10.625000,0.819149
41,(Kingambit),(Pelipper),0.375000,0.265625,0.250000,0.666667,2.509804,0.150391,2.203125,0.962500
8,(Amoonguss),(Pelipper),0.437500,0.265625,0.234375,0.535714,2.016807,0.118164,1.581731,0.896296
9,(Pelipper),(Amoonguss),0.265625,0.437500,0.234375,0.882353,2.016807,0.118164,4.781250,0.686525
